In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import shutil
import random
from pathlib import Path
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import requests
from tqdm import tqdm
from urllib.parse import urlparse
import numpy as np



In [3]:
from mtrain.experiment import run_experiments, cartesian_dict

In [12]:

DS = Path("/Users/hariomnarang/Desktop/personal/roads/datasets")
EXP_BASE = DS / "T009-unet"

EXP_BASE.mkdir(parents=True, exist_ok=True)

EXP_CONF = {
  "DATA_COUNT": ["5000"],
  "TILE_SIZE": ["20", "30", "40", "50", "60"],
  "BATCH_SIZE": ["2"],
  "FINE_TUNE_EPOCHS": ["15"],
  "FIT_EPOCHS": ["5"],
}
EXPERIMENTS = cartesian_dict("s009-unet", EXP_CONF)

EXTRA_ENV = {
  "DRIVE_BASE": str(EXP_BASE),
  "TACO_DIR": "/Users/hariomnarang/Desktop/personal/TACO/data",
  "WORKING_DATA_DIR": "/tmp/t008"
}

LOG_DIR = EXP_BASE / "logs"
LOG_DIR.mkdir(parents=True, exist_ok=True)

In [28]:
! ls '/Users/hariomnarang/Desktop/personal/roads/datasets/T009-unet/garbage/experiments/s009-unet-DATA_COUNT=100-TILE_SIZE=20-BATCH_SIZE=16-FINE_TUNE_EPOCHS=15-FIT_EPOCHS=5/log'

export.pkl  history.csv


In [17]:
Path(EXTRA_ENV["DRIVE_BASE"])

PosixPath('/Users/hariomnarang/Desktop/personal/roads/datasets/T009-unet')

In [10]:
run_experiments(
  EXPERIMENTS, LOG_DIR, ["uv", "run", "../scripts/colab/train_unet.py"], extra_env=EXTRA_ENV
)


🚀 Running [s009-unet-DATA_COUNT=5000-TILE_SIZE=20-BATCH_SIZE=2-FINE_TUNE_EPOCHS=15-FIT_EPOCHS=5] (attempt 1/1)
log file: /Users/hariomnarang/Desktop/personal/roads/datasets/T009-unet/logs/20260129_115230_s009-unet-DATA_COUNT=5000-TILE_SIZE=20-BATCH_SIZE=2-FINE_TUNE_EPOCHS=15-FIT_EPOCHS=5.log
running command:  PYTHONUNBUFFERED='1' DATA_COUNT='5000' TILE_SIZE='20' BATCH_SIZE='2' FINE_TUNE_EPOCHS='15' FIT_EPOCHS='5' name='s009-unet-DATA_COUNT=5000-TILE_SIZE=20-BATCH_SIZE=2-FINE_TUNE_EPOCHS=15-FIT_EPOCHS=5' PROJECT_CODE='s009-unet-DATA_COUNT=5000-TILE_SIZE=20-BATCH_SIZE=2-FINE_TUNE_EPOCHS=15-FIT_EPOCHS=5' DRIVE_BASE='['/Users/hariomnarang/Desktop/personal/roads/datasets/T009-unet']' TACO_DIR='/Users/hariomnarang/Desktop/personal/TACO/data' WORKING_DATA_DIR='/tmp/t008' uv run ../scripts/colab/train_unet.py


KeyboardInterrupt: 